In [4]:
import pickle
from scipy import stats
import pandas as pd
import numpy as np
import pickle5 as pickle
from sklearn.ensemble import RandomForestRegressor
from treeinterpreter import treeinterpreter as ti
import networkx as nx
import seaborn as sns
import stringdb
import sklearn

big_mat = []
for n in [0,0.25,0.75,1]:
    for m in [0,0.25,0.75,1]:
        big_mat.append([m,n])
                
def tree_to_df(reg_tree, feature_names):
    tree_ = reg_tree.tree_
    feature_name = [
        feature_names[i] if i != sklearn.tree._tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    
    def recurse(node, row, ret):
        if tree_.feature[node] != sklearn.tree._tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            # Add rule to row and search left branch
            row[-1].append(name + " <= " +  str(threshold))
            recurse(tree_.children_left[node], row, ret)
            # Add rule to row and search right branch
            row[-1].append(name + " > " +  str(threshold))
            recurse(tree_.children_right[node], row, ret)
        else:
            # Add output rules and start a new row
            label = tree_.value[node]
            ret.append("return " + str(label[0][0]))
            row.append([])
    
    # Initialize
    rules = [[]]
    vals = []
    
    # Call recursive function with initial values
    recurse(0, rules, vals)
    
    # Convert to table and output
    df = pd.DataFrame(rules).dropna(how='all')
    df['Return'] = pd.Series(vals)
    return df


col_ = ['ABL1', 'ABL2', 'AKT_i', 'APC', 'AXIN1', 'BAD', 'BAK1', 'BAX', 'BCL2', 'BCLXL', 'BID', 'BIM', 'CASP3', 'CASP8', 'CASP9', 'CDKN2A', 'CEBPB', 'CREB1', 'CTNNB1', 'CYCLIN_D_c', 'CYCS', 'DC', 'DLL_i', 'DVL1', 'E2F1', 'EGF', 'EGFR', 'EIF4E', 'ERK_i', 'ES', 'ESR1', 'FADD', 'FAS', 'FASLG', 'FOXO1', 'FOXO3', 'FZD3', 'GATA3', 'GRB2', 'GRB7', 'GSK3B', 'HER2', 'HER3', 'HES1', 'HEY1', 'HRAS', 'HSP90AA1', 'IGF1', 'IGF1R', 'IL1B', 'IL6', 'IL6R', 'IL8', 'INS', 'INSR', 'IRS1', 'IVEBP1', 'JAK2', 'JUN', 'KMT2D', 'KRT14', 'LATS1', 'LCK', 'MAPK8', 'MCL1', 'MDM2', 'MEK_i', 'MTORC1_c', 'MTORC2_c', 'MYC', 'MYOD1', 'MYOG', 'NCID', 'NEDD4L', 'NF1', 'NF2', 'NFKB1', 'NOS3', 'NOTCH1', 'NRG1', 'P21', 'P53', 'PAK1', 'PAK2', 'PAX7', 'PDPK1', 'PG', 'PGR', 'PI3K_c', 'PIM', 'PIP3', 'PPARG', 'PRB', 'PRKAA2', 'PRKACA', 'PTEN', 'RAC1', 'RAF1', 'RAS_i', 'RELA', 'RHEB', 'RND1', 'S6K1', 'SGK1', 'SOS1', 'SRC', 'STAT1', 'STAT3', 'STAT5', 'STK11', 'TCF4', 'TEAD1', 'TRADD', 'TSC2', 'TWIST1', 'WNT1', 'YAP1']
ONCO_=pd.read_excel(open('file/GRN.xls', 'rb'),sheet_name= "Node_description").query("types == 'ONCO'")["model_name (alias)"].drop_duplicates().tolist()

# 1. Prepare the perturbed protein activities as an input

In [5]:
name_p = []
list_=[]

for pert_node in ["PTEN"]:
    
    df = pd.read_csv("result/M468_GoF_"+pert_node+".csv.xz").iloc[:,1:]
    name_ = pert_node+"_1"

    activity = []
    for mt in big_mat:

        d1,d2 = mt[0],mt[1]
        d_name = str(d1)+"_"+str(d2)
        d_name2 = str(d1)+" vs. "+str(d2)
        atr_adj = 10000 - 500

        df_tmp = df.query("dose == @d_name2").query("variable >= @atr_adj")
        df_tmp_2 = df_tmp[["Gene","variable","value"]].groupby(["Gene"]).mean().reset_index()
        df_tmp_2["dose"] = d_name2
        df_tmp_2["drug_pair"] = name_

        activity.append(df_tmp_2)

    df_sum = pd.concat(activity)[["Gene","dose","value"]].pivot(index='Gene', columns='dose', values='value').reset_index()
    list_.append(df_sum)
    name_p.append(pert_node+"1")

pp_list = []
for i in list_:
    ddd1 = i.T.iloc[1:,:]/100
    ddd = ddd1.mean(axis=0).to_numpy()
    pp_list.append(ddd)
    
Input2 = np.array(pp_list)
Input2

array([[0.      , 0.      , 0.      , 1.      , 1.      , 0.514375,
        1.      , 0.244375, 0.      , 0.      , 1.      , 0.94375 ,
        0.7575  , 1.      , 0.7575  , 1.      , 1.      , 0.      ,
        0.      , 0.      , 1.      , 1.      , 0.7     , 0.      ,
        0.      , 0.36    , 0.478125, 0.      , 0.236875, 0.36    ,
        0.      , 0.875   , 0.      , 0.      , 1.      , 0.94375 ,
        0.      , 0.      , 0.      , 0.99    , 1.      , 0.165   ,
        0.165   , 0.7525  , 0.7     , 0.      , 0.      , 0.05375 ,
        0.478125, 0.      , 0.      , 0.      , 0.478125, 0.53    ,
        0.99    , 1.      , 1.      , 1.      , 0.675625, 0.      ,
        0.478125, 0.      , 0.      , 0.675625, 0.753125, 0.      ,
        0.236875, 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.7     , 1.      , 1.      , 1.      , 0.7     , 0.      ,
        0.7     , 0.43    , 0.      , 0.      , 0.      , 0.      ,
        1.      , 0.      , 0.5     , 0.      , 

# 2. Predict synergy score from input

In [6]:

param_list =[{'ccp_alpha': 0.5,
  'max_depth': 4,
  'max_samples': 0.7,
  'min_samples_leaf': 16,
  'min_samples_split': 32,
  'n_estimators': 2000},
 {'ccp_alpha': 0.5,
  'max_depth': 5,
  'max_samples': 0.7,
  'min_samples_leaf': 16,
  'min_samples_split': 24,
  'n_estimators': 2000},
 {'ccp_alpha': 0.5,
  'max_depth': 5,
  'max_samples': 0.7,
  'min_samples_leaf': 16,
  'min_samples_split': 16,
  'n_estimators': 2000},
 {'ccp_alpha': 0.7,
  'max_depth': 4,
  'max_samples': 0.7,
  'min_samples_leaf': 16,
  'min_samples_split': 24,
  'n_estimators': 2000},
 {'ccp_alpha': 0.5,
  'max_depth': 3,
  'max_samples': 0.7,
  'min_samples_leaf': 16,
  'min_samples_split': 32,
  'n_estimators': 2000}]

In [7]:
res_pred_list = []
cnt = 0
for cnt in range(len(param_list)):
    
    best_model = pickle.load(open('result/refitted_RF_'+str(cnt), 'rb'))
    pred_val = best_model.predict(Input2)
    res_pred = pd.DataFrame(np.array([pred_val])).T
    res_pred["model"] = res_pred.index
    res_pred["index"] = name_p
    res_pred_list.append(res_pred)
    cnt += 1

score_pert = pd.concat(res_pred_list).rename({0:"pred"},axis=1).groupby(["index"]).mean().reset_index().iloc[:,:2]

# 3. Get feature contributions

In [8]:
#loop trhough five model

res_pred_list = []
res_con_list = []

for cnt in range(len(param_list)):

    RF = RandomForestRegressor()
    best_model = pickle.load(open('result/refitted_RF_'+str(cnt), 'rb'))
    prediction, bias, contributions = ti.predict(best_model ,Input2)        

    res_con_list.append(contributions)
    res_pred_list.append(prediction)

    cnt += 1


In [9]:
df_list = []
for i in range(len(param_list)):
    pd_ = pd.DataFrame(res_con_list[i],columns=col_)
    pd_["model"] = i
    pd_["index"] = name_p
    df_list.append(pd_)

df_ave = pd.concat(df_list).groupby(["index"]).mean().drop(['model'], axis=1).reset_index()

In [10]:
df_ave['index'] = pd.Categorical(
    df_ave['index'], 
    categories=name_p, 
    ordered=True
)

In [11]:
df_ave1 = df_ave.sort_values('index').melt(id_vars="index")
df_ave1

,index,variable,value
0,PTEN1,ABL1,0.000494
1,PTEN1,ABL2,-0.000005
2,PTEN1,AKT_i,0.046221
3,PTEN1,APC,-0.005107
4,PTEN1,AXIN1,-0.009113
...,...,...,...
112,PTEN1,TRADD,0.021432
113,PTEN1,TSC2,0.083989
114,PTEN1,TWIST1,-0.004629
115,PTEN1,WNT1,0.005842


# 4. Get the feature contributions of the downstream proteins within 5 neighbor from drug target proteins

In [12]:
df_fil = df_ave1.query("index == 'PTEN1'").sort_values(by="value",ascending=False)
var_sel = df_fil.variable.tolist()
var_index = [col_.index(_) for _ in var_sel]

In [13]:
edge_list=[]
df_ = pd.read_excel(open('file/GRN.xls', 'rb'),sheet_name= "MDAMB468_complete")
df_index = df_["target"].to_frame()
df_index["index"] = df_index.index
index_gcn = dict(df_index.values)

all_df = []
for i in range(df_.shape[0]):
    target = df_.iloc[i,0]
    source = df_.iloc[i,1]
    source_2 = source.replace('and', '').replace('or', '').replace('not', '').replace('(', '').replace(')', '')
    source_3 = source_2.split("  ")
    source_4 = [_.replace(" ","") for _ in source_3]
    df_concat = pd.DataFrame([target],columns=["target"])
    dup_df = pd.concat([df_concat]*len(source_4))
    dup_df["source"] = source_4
    all_df.append(dup_df)

df_concat_all = pd.concat(all_df)

In [14]:
edge_df = df_concat_all.drop_duplicates()
G=nx.from_pandas_edgelist(edge_df, 'source', 'target',create_using=nx.DiGraph())

In [15]:
kn1 = nx.single_source_shortest_path_length(G,"MEK_i", cutoff=10)
df2_list1 = pd.DataFrame(kn1.items(),columns=["gene","KN"]).query("gene != ''").query("KN != 0").query("KN <= 5").gene.tolist()

kn2 = nx.single_source_shortest_path_length(G,"STAT3", cutoff=10)
df2_list2 = pd.DataFrame(kn2.items(),columns=["gene","KN"]).query("gene != ''").query("KN != 0").query("KN <= 5").gene.tolist()

kn3 = nx.single_source_shortest_path_length(G,"PTEN", cutoff=10)
df2_list3 = pd.DataFrame(kn3.items(),columns=["gene","KN"]).query("gene != ''").query("KN != 0").query("KN <= 5").gene.tolist()

df2_list = list(set(df2_list1+df2_list2+df2_list3))

In [16]:
index_sel = name_p.index("PTEN1")
array_ = Input2[index_sel][var_index]
df_fil["steady_state"] = array_

df3 = df_fil.iloc[:,1:]
df3_coef = df3.iloc[:,[0,1]]
df3_coef["type"] = "Averaged Feature contribution"

df3_ste = df3.iloc[:,[0,2]].rename({"steady_state":"value"},axis=1)
df3_ste["type"] = "Protein Activities (Inputs)"
df3_list = df3_ste.query("value == 0").variable.tolist()

dff_concat = pd.concat([df3_ste,df3_coef])
final_list = dff_concat.variable.unique().tolist()

In [17]:
# threshold
value_list = []

for cnt in range(len(param_list)):
    best_model = pickle.load(open('result/refitted_RF_'+str(cnt), 'rb'))

    cor_list = []
    for tree in best_model.estimators_:

        r =  tree_to_df(tree, feature_names=col_)

        r2 = r.drop(['Return'], axis=1).melt().dropna().iloc[:,1:]
        r3 = r2['value'].str.split(' ', expand=True)
        r4=r3[[0,2]]

        cor_list.append(r4)

    df_1 = pd.concat(cor_list)
    value_list.append(df_1)


In [18]:
df_2 = pd.concat(value_list)
df_2.columns = ["variable","value"]
df_2["value"] = df_2["value"].astype(float)
df_3 = df_2.groupby(["variable"]).mean().reset_index().query("variable == @final_list")
df_3["type"] = "Averaged RF threshold"

In [19]:
df_4 = pd.concat([dff_concat,df_3]).query("variable == @df2_list")
df_5 = df_4.pivot(index='variable', columns='type', values="value").reset_index()
df_5["cat"]=np.where(df_5["variable"].isin(ONCO_),"ONCO","TMR")
df_5["QA1"] = np.where((df_5["Protein Activities (Inputs)"] < df_5["Averaged RF threshold"]) & (df_5["Averaged Feature contribution"] <= 0) & (df_5["cat"] == "TMR"),"N",
                       np.where((df_5["Protein Activities (Inputs)"] > df_5["Averaged RF threshold"]) & (df_5["Averaged Feature contribution"] <= 0) & (df_5["cat"] == "TMR"),"N",
                               np.where((df_5["Protein Activities (Inputs)"] < df_5["Averaged RF threshold"]) & (df_5["Averaged Feature contribution"] <= 0) & (df_5["cat"] == "ONCO"),"N",
                                       np.where((df_5["Protein Activities (Inputs)"] > df_5["Averaged RF threshold"]) & (df_5["Averaged Feature contribution"] >= 0) & (df_5["cat"] == "ONCO"),"N","Y"))))


In [20]:
df_qual_list = df_5.query("QA1 == 'Y'").variable.tolist()
df_qual = df_5.query("QA1 == 'Y'")

In [21]:
GRN = pd.read_excel(open('file/GRN.xls', 'rb'),sheet_name= "Node_description")
hgnc = GRN.query("`model_name (alias)` == @df_qual_list")['gene_name (HGNC)'].tolist()
string_ids = stringdb.get_string_ids(hgnc)
enrichment_df = stringdb.get_enrichment(string_ids.queryItem)
p = ["PI3K-Akt signaling pathway","FoxO signaling pathway","ErbB signaling pathway","JAK-STAT signaling pathway","mTOR signaling pathway","Estrogen signaling pathway","Insulin signaling pathway","Prolactin signaling pathway","TNF signaling pathway","Hippo signaling pathway","Ras signaling pathway",
    "p53 signaling pathway","Wnt signaling pathway","Apoptosis"]
pp = enrichment_df.query("category == 'KEGG'").query("description == @p")

# 5. Summarizing the feautre contribution of MCF7 AKT-GOF mutation

In [22]:
cc_list = []
for i in range(pp.shape[0]):
    sel = pp.iloc[i,:].tolist()

    num_node = sel[2]
    component = sel[5]
    path_name = sel[9]

    hgnc_en = component.split(",")
    model_en = list(GRN.query("`gene_name (HGNC)` == @hgnc_en")["model_name (alias)"].unique())
   
    df_11 = df_qual.query("variable == @model_en")[["variable","Averaged Feature contribution"]]
    df_11["KEGG"] = path_name
    df_11["cc"] = component

    cc_list.append(df_11)

In [26]:
df_=pd.concat(cc_list)
df_33=df_.groupby(["KEGG"]).sum().reset_index().sort_values(by="Averaged Feature contribution",ascending=False)
top_ = df_33.head(n=5).KEGG.tolist()
sel_prot = pd.concat(cc_list).query("KEGG == @top_").sort_values('Averaged Feature contribution',ascending=False)
sel_prot_list = sel_prot.groupby(["KEGG"]).head(5).variable.unique().tolist()
sel_prot_df = sel_prot.groupby(["KEGG"]).head(5)
df_4.query("variable == @sel_prot_list").pivot(index='variable', columns='type', values='value')

type,Averaged Feature contribution,Averaged RF threshold,Protein Activities (Inputs)
variable,,,
AKT_i,0.046221,0.497050,0.00000
BAK1,0.122243,0.393231,1.00000
BID,0.186873,0.294166,1.00000
CASP3,0.089259,0.230153,0.75750
CASP9,0.073385,0.189492,0.75750
FOXO3,0.451055,0.634580,0.94375
MTORC1_c,0.519854,0.274811,0.00000
MTORC2_c,0.315409,0.610052,0.00000
NFKB1,0.391327,0.734206,0.70000
